In [ ]:
#best_model2.pth
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import pandas as pd

# ✅ 하이퍼파라미터 설정
num_epochs = 1000
batch_size = 32
learning_rate = 0.001
dropout_rate = 0.3
early_stop_threshold = 0.95  # 얼리 스탑 기준 검증 정확도

# ✅ 데이터 전처리 및 증강
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드 (새로운 데이터 구조 반영)
data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\brand_data"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# ✅ 데이터셋 분할 (10% 테스트, 나머지 90%를 학습 및 검증 데이터로 활용)
test_size = int(0.1 * len(dataset))
train_val_size = len(dataset) - test_size
dataset_train_val, dataset_test = random_split(dataset, [train_val_size, test_size])

# ✅ 학습 및 검증 데이터 분할 (80% 학습, 20% 검증)
train_size = int(0.8 * len(dataset_train_val))
valid_size = len(dataset_train_val) - train_size
train_dataset, valid_dataset = random_split(dataset_train_val, [train_size, valid_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

# ✅ 모델 초기화
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

# ✅ 드롭아웃 레이어 추가
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, len(dataset.classes)),  # 클래스 수 자동 설정
    nn.Dropout(dropout_rate)  # 드롭아웃 추가
)

# ✅ 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# ✅ 결과 저장 폴더 설정
model_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/model"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/csv"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

best_model2_path = os.path.join(model_dir, "best_model2.pth")
csv_path = os.path.join(csv_dir, "resnet18_epoch_results.csv")

# ✅ 학습 및 검증 함수
def train_and_validate(model, train_loader, valid_loader, criterion, optimizer, num_epochs, csv_path, early_stop_threshold):
    best_valid_accuracy = 0.0
    best_epoch = 0
    best_train_loss = 0.0
    best_train_accuracy = 0.0
    best_valid_loss = 0.0

    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0

        # ✅ 학습 단계
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = train_correct / total_train
        train_loss /= len(train_loader)

        # ✅ 검증 단계
        model.eval()
        valid_loss = 0.0
        valid_correct = 0
        total_valid = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                valid_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                valid_correct += (predicted == labels).sum().item()

        valid_accuracy = valid_correct / total_valid
        valid_loss /= len(valid_loader)

        # ✅ 결과 출력
        print(f"📢 Epoch [{epoch+1}/{num_epochs}] | "
            f"📉 Train Loss: {train_loss:.4f} | "
            f"🎯 Train Accuracy: {train_accuracy:.4f} | "
            f"📉 Valid Loss: {valid_loss:.4f} | "
            f"🎯 Valid Accuracy: {valid_accuracy:.4f}")
        
        # ✅ 손실 및 정확도 저장
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)

        # ✅ 최고 검증 정확도 모델 저장
        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            best_epoch = epoch + 1
            best_train_loss = train_loss
            best_train_accuracy = train_accuracy
            best_valid_loss = valid_loss

            torch.save(model.state_dict(), best_model2_path)
            print(f"🎯 새로운 최고 모델 저장됨! (Epoch {best_epoch}, Accuracy: {best_valid_accuracy:.4f})")

        # ✅ 얼리 스탑 조건 확인
        if valid_accuracy >= early_stop_threshold:
            print(f"🚀 얼리 스탑! 검증 정확도가 {early_stop_threshold * 100:.2f}%를 초과하여 학습을 중단합니다.")
            break

    # ✅ 결과를 CSV 파일로 저장
    df = pd.DataFrame({
        'Epoch': range(1, epoch + 2),
        'Train Loss': train_losses,
        'Valid Loss': valid_losses,
        'Train Accuracy': train_accuracies,
        'Valid Accuracy': valid_accuracies
    })
    df.to_csv(csv_path, index=False)
    print(f"📄 학습 결과 CSV 저장됨: {csv_path}")

# ✅ 모델 학습
train_and_validate(model, train_loader, valid_loader, criterion, optimizer, num_epochs, csv_path, early_stop_threshold)

c:\Users\user\anaconda3\envs\classification\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\classification\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


📢 Epoch [1/1000] | 📉 Train Loss: 1.9064 | 🎯 Train Accuracy: 0.4353 | 📉 Valid Loss: 1.5526 | 🎯 Valid Accuracy: 0.5797
🎯 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 0.5797)
📢 Epoch [2/1000] | 📉 Train Loss: 1.4194 | 🎯 Train Accuracy: 0.5634 | 📉 Valid Loss: 1.1245 | 🎯 Valid Accuracy: 0.6719
🎯 새로운 최고 모델 저장됨! (Epoch 2, Accuracy: 0.6719)
📢 Epoch [3/1000] | 📉 Train Loss: 1.2785 | 🎯 Train Accuracy: 0.5961 | 📉 Valid Loss: 1.0513 | 🎯 Valid Accuracy: 0.6947
🎯 새로운 최고 모델 저장됨! (Epoch 3, Accuracy: 0.6947)
📢 Epoch [4/1000] | 📉 Train Loss: 1.1793 | 🎯 Train Accuracy: 0.6246 | 📉 Valid Loss: 0.7607 | 🎯 Valid Accuracy: 0.7896
🎯 새로운 최고 모델 저장됨! (Epoch 4, Accuracy: 0.7896)
📢 Epoch [5/1000] | 📉 Train Loss: 1.0700 | 🎯 Train Accuracy: 0.6516 | 📉 Valid Loss: 0.9038 | 🎯 Valid Accuracy: 0.7570
📢 Epoch [6/1000] | 📉 Train Loss: 1.0203 | 🎯 Train Accuracy: 0.6658 | 📉 Valid Loss: 0.7425 | 🎯 Valid Accuracy: 0.7918
🎯 새로운 최고 모델 저장됨! (Epoch 6, Accuracy: 0.7918)
📢 Epoch [7/1000] | 📉 Train Loss: 0.9198 | 🎯 Train Accuracy: 0.6891 | 📉 Va